<a href="https://colab.research.google.com/github/alinaalborova/sentence-transformers/blob/master/sentence_embeddings_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.6/dist-packages (0.2.5)


In [2]:
!pip install h5py pyyaml

In [3]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'1.15.0'

In [13]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

FileNotFoundError: ignored

In [9]:
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']
sentence_embeddings = model.encode(sentences)

AttributeError: ignored

In [0]:
for sentence, embedding in zip(sentences, sentence_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", len(embedding), embedding[:10])
    print("")

In [0]:
sentence_embeddings = model.encode(['Мама мыла раму'])

In [0]:
rus_sentences = ['мама мыла раму', 'рама мыла маму']
rus_sentences_embeddings = model.encode(rus_sentences)
for sentence, embedding in zip(rus_sentences, rus_sentences_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", len(embedding), embedding[:10])
    print("")

In [0]:
de_sentences = ['Mein Hund ist gross', 'Meine Katze ist gross', 'Mein Hund ist klein', 'Dein Hund ist klein', 'Dein Hund ist gross', 'Meine Katze ist klein', 'Deine Katze ist klein', 'Deine Katze ist gross']
de_sentences_embeddings = model.encode(de_sentences)
for sentence, embedding in zip(de_sentences, de_sentences_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", len(embedding), embedding[:10])
    print("")    

# FAISS Installation

Sadly, it can be painfull :(

In according to answer from SO: https://stackoverflow.com/questions/47967252/installing-faiss-on-google-colaboratory

In [14]:
#!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
#!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!wget https://anaconda.org/pytorch/faiss-gpu/1.2.1/download/linux-64/faiss-gpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!tar xvjf faiss-gpu-1.2.1-py36_cuda9.0.176_1.tar.bz2

--2020-02-13 13:06:10--  https://anaconda.org/pytorch/faiss-gpu/1.2.1/download/linux-64/faiss-gpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
Resolving anaconda.org (anaconda.org)... 104.17.93.24, 104.17.92.24, 2606:4700::6811:5d18, ...
Connecting to anaconda.org (anaconda.org)|104.17.93.24|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5a15c9cc393ec612061f033a/5ac54beae586bb11ec0ace46?response-content-disposition=attachment%3B%20filename%3D%22faiss-gpu-1.2.1-py36_cuda9.0.176_1.tar.bz2%22%3B%20filename%2A%3DUTF-8%27%27faiss-gpu-1.2.1-py36_cuda9.0.176_1.tar.bz2&response-content-type=application%2Fx-tar&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Expires=60&X-Amz-Date=20200213T130611Z&X-Amz-SignedHeaders=host&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEKT%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJHMEUCICwQkNzsG37VNuZKlZNT0dwNKgCGwxiRITKMkIuySDXsAiEAyM1V8UVxPrarHJpZnJPTvfB9ne%2Bl4hQipW7be7%2F3p0wqtAMIXBAAGgw0NTU4NjQwOTgzNzg

In [15]:
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

In [0]:
import faiss

Now time for downloading datasets

In [0]:
import math
import logging
from datetime import datetime
from pathlib import Path
import os
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import joblib
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import models, losses
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import *
from sentence_transformers.util import batch_to_device
from sentence_transformers.readers.InputExample import InputExample

In [0]:
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

# Tatoeba manipulation

In [19]:
!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/moses/de-ru.txt.zip
!unzip de-ru.txt.zip -d ./tatoeba/

--2020-02-13 13:06:28--  https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/moses/de-ru.txt.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3873926 (3.7M) [application/zip]
Saving to: ‘de-ru.txt.zip’

de-ru.txt.zip       100%[===================>]   3.69M  1.84MB/s    in 2.0s    

2020-02-13 13:06:32 (1.84 MB/s) - ‘de-ru.txt.zip’ saved [3873926/3873926]

Archive:  de-ru.txt.zip
  inflating: ./tatoeba/README        
  inflating: ./tatoeba/LICENSE       
  inflating: ./tatoeba/Tatoeba.de-ru.de  
  inflating: ./tatoeba/Tatoeba.de-ru.ru  
  inflating: ./tatoeba/Tatoeba.de-ru.xml  


In [20]:
!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/moses/de-en.txt.zip
!unzip de-en.txt.zip -d ./tatoeba/


--2020-02-13 13:06:37--  https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/moses/de-en.txt.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9957647 (9.5M) [application/zip]
Saving to: ‘de-en.txt.zip’

de-en.txt.zip       100%[===================>]   9.50M  3.68MB/s    in 2.6s    

2020-02-13 13:06:41 (3.68 MB/s) - ‘de-en.txt.zip’ saved [9957647/9957647]

Archive:  de-en.txt.zip
replace ./tatoeba/README? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ./tatoeba/LICENSE? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace ./tatoeba/LICENSE? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
  inflating: ./tatoeba/Tatoeba.de-en.de  
  inflating: ./tatoeba/Tatoeba.de-en.en  
  inflating: ./tatoeba/Tatoeba.de-en.xml  


In [0]:
class TatoebaReader:
    """Reads in a plain text file, in which every line contains one 
    sentence."""
    def __init__(self, file_path: Path):
        self.file_path = file_path

    def get_examples(self, limit = 500):
        examples = []        
        with open(self.file_path) as fin:
            for i, line in enumerate(fin.readlines()):
                examples.append(InputExample(guid=i, texts=[line], label=0))
                if limit == -1 :
                  continue
                elif i > limit:
                  break                
        return examples

TATOEBA_PATH = Path("./tatoeba/")

In [0]:

# Use BERT for mapping tokens to embeddings
# handle the downloading and caching for you:
word_embedding_model = models.BERT('bert-base-multilingual-cased')

In [0]:
def children(m):
    return m if isinstance(m, (list, tuple)) else list(m.children())


def set_trainable_attr(m, b):
    m.trainable = b
    for p in m.parameters():
        p.requires_grad = b


def apply_leaf(m, f):
    c = children(m)
    if isinstance(m, nn.Module):
        f(m)
    if len(c) > 0:
        for l in c:
            apply_leaf(l, f)


def set_trainable(l, b):
    apply_leaf(l, lambda m: set_trainable_attr(m, b))

In [24]:
set_trainable(word_embedding_model.bert.embeddings.word_embeddings, False)
print(word_embedding_model.bert.embeddings.word_embeddings.weight.requires_grad)
print(word_embedding_model.bert.embeddings.position_embeddings.weight.requires_grad)

False
True


In [0]:

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [0]:
batch_size = 16
TATOEBA_PATH = '/content/tatoeba'

In [0]:
lang_1 = 'ru'
lang_2 = 'de'

In [0]:

def evaluate_language_pair(model, pair_name="de-en", batch_size=32):
    lang_1, lang_2 = pair_name.split("-")
    reader_1 = TatoebaReader(os.path.join(TATOEBA_PATH, f"Tatoeba.{pair_name}.{lang_1}"))
    ds_1 = SentencesDataset(reader_1.get_examples(), model=model)
    loader_1 = DataLoader(
        ds_1, shuffle=False, batch_size=batch_size, 
        collate_fn=model.smart_batching_collate)
    reader_2 = TatoebaReader(os.path.join(TATOEBA_PATH, f"Tatoeba.{pair_name}.{lang_2}"))    
    ds_2 = SentencesDataset(reader_2.get_examples(), model=model)
    loader_2 = DataLoader(
        ds_2, shuffle=False, batch_size=batch_size, 
        collate_fn=model.smart_batching_collate)
    
    model.eval()
    emb_1, emb_2 = [], []
    with torch.no_grad():
        for batch in loader_1:
            emb_1.append(model(
                batch_to_device(batch, "cuda")[0][0]
            )['sentence_embedding'])
        for batch in loader_2:
            emb_2.append(model(
                batch_to_device(batch, "cuda")[0][0]
            )['sentence_embedding'])
    emb_1 = torch.cat(emb_1).cpu().numpy()
    emb_2 = torch.cat(emb_2).cpu().numpy()
    
    idx_1 = faiss.IndexFlatL2(emb_1.shape[1])
    faiss.normalize_L2(emb_1)
    idx_1.add(emb_1)
    idx_2 = faiss.IndexFlatL2(emb_2.shape[1])
    faiss.normalize_L2(emb_2)
    idx_2.add(emb_2)
    
    results = []
    _, match = idx_2.search(x=emb_1, k=1)
    results.append((
        lang_1, lang_2,
        np.sum(match[:, 0] == np.arange(len(emb_1))),
        len(emb_1)
    ))
    _, match = idx_1.search(x=emb_2, k=1)
    results.append((
        lang_2, lang_1,
        np.sum(match[:, 0] == np.arange(len(emb_2))),
        len(emb_2)
    ))
    return results

In [0]:
PAIRS = ["de-en", "de-ru"]

In [30]:
results = []
for pair in PAIRS:
    results += evaluate_language_pair(model, pair_name=pair, batch_size=50)
df_baseline_mean = pd.DataFrame(results, columns=["from", "to", "correct", "total"])
df_baseline_mean

,from,to,correct,total
0,de,en,210,502
1,en,de,194,502
2,de,ru,200,502
3,ru,de,184,502


In [0]:
tf.__version__

# Fine Tuning 

Firstly, just download datasets for fine-tuning.

In [0]:
import os
folder_path = './datasets/'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)


In [32]:
import urllib.request
import zipfile
print('Beginning download of datasets')

datasets = ['AllNLI.zip', 'stsbenchmark.zip', 'wikipedia-sections-triplets.zip']
server = "https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/datasets/"

for dataset in datasets:
    print("Download", dataset)
    url = server+dataset
    dataset_path = os.path.join(folder_path, dataset)
    urllib.request.urlretrieve(url, dataset_path)

    print("Extract", dataset)
    with zipfile.ZipFile(dataset_path, "r") as zip_ref:
        zip_ref.extractall(folder_path)
    os.remove(dataset_path)


print("All datasets downloaded and extracted")

Beginning download of datasets
Download AllNLI.zip
Extract AllNLI.zip
Download stsbenchmark.zip
Extract stsbenchmark.zip
Download wikipedia-sections-triplets.zip
Extract wikipedia-sections-triplets.zip
All datasets downloaded and extracted


In [0]:

# Read the dataset
batch_size = 16
nli_reader = NLIDataReader('./datasets/AllNLI')
sts_reader = STSDataReader('./datasets/stsbenchmark')
train_num_labels = nli_reader.get_num_labels()

In [0]:
# Convert the dataset to a DataLoader ready for training
logging.info("Read AllNLI train dataset")
train_data = SentencesDataset(nli_reader.get_examples('train.gz', max_examples=100000), model=model)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
train_loss = losses.SoftmaxLoss(
    model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=train_num_labels)


In [0]:
joblib.dump(train_data, "allnli_train_dataset.jl")

In [0]:

logging.info("Read STSbenchmark dev dataset")
dev_data = SentencesDataset(examples=sts_reader.get_examples('sts-dev.csv'), model=model)
dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=batch_size)
evaluator = EmbeddingSimilarityEvaluator(dev_dataloader)

In [0]:
joblib.dump(dev_data, "sts_dev_dataset.jl")

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#checkpoint_path = "gdrive/embeddings_01/cp.ckpt"
#checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
#cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
#                                                 save_weights_only=True,
#                                                 verbose=1)
model_save_path = 'drive/My Drive/models/training_nli_bert-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

### PAUSE HERE!

In [5]:
# Configure the training
num_epochs = 1

warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path
          )

NameError: ignored

# After fine-tuning

In [0]:
#model = SentenceTransformer('/content/output/training_nli_bert-2020-01-13_14-53-24')
#model = SentenceTransformer('/content/output/training_nli_bert-2020-01-23_17-06-00')
#model = SentenceTransformer('/content/gdrive/output/training_nli_bert-2020-02-09_11-48-02')
model = SentenceTransformer('/content/drive/My Drive/models/training_nli_bert-2020-02-09_13-04-35')

In [0]:
results = []
for pair in PAIRS:
    results += evaluate_language_pair(model, pair_name=pair, batch_size=50)
df_finetuned = pd.DataFrame(results, columns=["from", "to", "correct", "total"])

In [38]:
df_finetuned

,from,to,correct,total
0,de,en,145,502
1,en,de,120,502
2,de,ru,131,502
3,ru,de,124,502


In [39]:
df_baseline_mean

,from,to,correct,total
0,de,en,210,502
1,en,de,194,502
2,de,ru,200,502
3,ru,de,184,502


In [0]:
word_embedding_model = models.BERT('bert-base-multilingual-cased')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=False,
                               pooling_mode_cls_token=True,
                               pooling_mode_max_tokens=False)

model_cls = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [46]:
results = []
for pair in PAIRS:
    results += evaluate_language_pair(model_cls, pair_name=pair, batch_size=50)
df_baseline_cls = pd.DataFrame(results, columns=["from", "to", "correct", "total"])
df_baseline_cls

,from,to,correct,total
0,de,en,57,502
1,en,de,104,502
2,de,ru,88,502
3,ru,de,123,502


In [0]:
word_embedding_model = models.BERT('bert-base-multilingual-cased')
# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=False,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=True)
model_max = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [48]:
results = []
for pair in PAIRS:
    results += evaluate_language_pair(model_max, pair_name=pair, batch_size=50)
df_baseline_max = pd.DataFrame(results, columns=["from", "to", "correct", "total"])
df_baseline_max

,from,to,correct,total
0,de,en,107,502
1,en,de,147,502
2,de,ru,122,502
3,ru,de,109,502


In [0]:
df_baseline_mean["err_mean"] = 1 - df_baseline_mean["correct"] / df_baseline_mean["total"]
df_baseline_max["err_max"] = 1 - df_baseline_max["correct"] / df_baseline_max["total"]
df_baseline_cls["err_cls"] = 1 - df_baseline_cls["correct"] / df_baseline_cls["total"]
df_finetuned["err_finetuned"] = 1 - df_finetuned["correct"] / df_finetuned["total"]

In [50]:
df_err = pd.concat([
    df.set_index(["from", "to"]).drop(["correct", "total"], axis=1)
    for df in (df_baseline_mean, df_baseline_max, df_baseline_cls, df_finetuned)
], axis=1)
df_err

,,err_mean,err_max,err_cls,err_finetuned
from,to,,,,
de,en,0.581673,0.786853,0.886454,0.711155
en,de,0.613546,0.707171,0.792829,0.760956
de,ru,0.601594,0.756972,0.824701,0.739044
ru,de,0.633466,0.782869,0.754980,0.752988
